## An OMOP Cohort: Patients Exposed to Amiodarone Hydrochloride > 30 Days

### Part II: Data Cleaning & Reshaping

An examination of de-identified longitudinal visit and procedure data (derived to OMOP from Medicare DeSYNPuf data, 2008-2010, by OHDSI) and associated ICD9CM diagnosis codes to analyze the incidence and prevalence of AEs associated with amiodarone toxicity to the lung, thyroid and liver.

Kyle P. Rasku MS BSN RN

In [1]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

Load data sets.

In [4]:
cohort_ds = pd.read_csv("amiodarone_exposed_patients.csv", parse_dates=["patient_dob"])
visits_ds = pd.read_csv("amiodarone_exposed_visits.csv", parse_dates=["treatment_start", "vstart", "vend"])
procedures_ds = pd.read_csv("amiodarone_exposed_procedures.csv", parse_dates=["drug_treatment_start", "vstart", "vend"])
cpt_codes = pd.read_csv("cpt_lookup.csv")
conditions_ds = pd.read_csv("conditions_all.csv", parse_dates=["treatment_start", "condition_start_date", "condition_end_date"])

Clean and reshape visits with diagnoses.

In [5]:
visits_ds.head()

,patient_id,treatment_start,treatment_period,vtype,visit_id,diagnosis_name,snomed_code,ICD9CODE,vstart,vend,vduration,vprovider,care_site
0,111598,2008-01-15,90,VISIT,5351372,Vomiting,422400008,78703,2008-01-28,2008-01-28,NaN,50058.0,11234.0
1,111598,2008-01-15,90,VISIT,5351372,Epidemic vertigo,186738001,38612,2008-01-28,2008-01-28,NaN,50058.0,11234.0
2,111598,2008-01-15,90,VISIT,5351385,Atrial fibrillation,49436004,42731,2008-01-28,2008-01-28,NaN,315573.0,11283.0
3,111598,2008-01-15,90,VISIT,5351385,Atrial fibrillation,49436004,42731,2008-01-28,2008-01-28,NaN,315573.0,11283.0
4,111598,2008-01-15,90,VISIT,5351385,Paroxysmal supraventricular tachycardia,67198005,4270,2008-01-28,2008-01-28,NaN,315573.0,11283.0


In [6]:
visit_diagnoses = visits_ds[["visit_id", "diagnosis_name", "snomed_code", "ICD9CODE"]].copy()
visit_diagnoses.columns = ["visit_id", "diagnosis", "snomed_code", "icd9_code"]
visit_diagnoses.head()

,visit_id,diagnosis,snomed_code,icd9_code
0,5351372,Vomiting,422400008,78703
1,5351372,Epidemic vertigo,186738001,38612
2,5351385,Atrial fibrillation,49436004,42731
3,5351385,Atrial fibrillation,49436004,42731
4,5351385,Paroxysmal supraventricular tachycardia,67198005,4270


In [9]:
grouped = visit_diagnoses.groupby(['visit_id'], as_index = False).agg({'diagnosis': lambda x: ';'.join(set(x)), 
                                                                'snomed_code': lambda x: ';'.join(set(x)),
                                                                'icd9_code': lambda x: ';'.join(set(x))})
long = {}
long[0] = grouped['diagnosis'].str.split(pat=";", expand=True)
long[1] = grouped['snomed_code'].str.split(pat=";",expand=True)
long[2] = grouped['icd9_code'].str.split(pat=";",expand=True)
wide = pd.concat(long, axis=1)
wide["visit_id"] = grouped["visit_id"].copy()

wide.columns = ["diagnosis_1", "diagnosis_2", "diagnosis_3", "diagnosis_4", "diagnosis_5", "diagnosis_6", "diagnosis_7", "diagnosis_8", 
                "diagnosis_9", "diagnosis_10", "diagnosis_11", "snomed_1", "snomed_2", "snomed_3", "snomed_4", "snomed_5", "snomed_6", 
                "snomed_7", "snomed_8", "snomed_9", "snomed_10", "snomed_11", "icd9_1", "icd9_2", "icd9_3", "icd9_4", "icd9_5", 
                "icd9_6", "icd9_7", "icd9_8", "icd9_9", "icd9_10", "icd9_11", "visit_id"]

wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5595 entries, 0 to 5594
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   diagnosis_1   5595 non-null   object
 1   diagnosis_2   4257 non-null   object
 2   diagnosis_3   2125 non-null   object
 3   diagnosis_4   1026 non-null   object
 4   diagnosis_5   439 non-null    object
 5   diagnosis_6   185 non-null    object
 6   diagnosis_7   85 non-null     object
 7   diagnosis_8   34 non-null     object
 8   diagnosis_9   10 non-null     object
 9   diagnosis_10  2 non-null      object
 10  diagnosis_11  1 non-null      object
 11  snomed_1      5595 non-null   object
 12  snomed_2      4257 non-null   object
 13  snomed_3      2125 non-null   object
 14  snomed_4      1026 non-null   object
 15  snomed_5      439 non-null    object
 16  snomed_6      185 non-null    object
 17  snomed_7      85 non-null     object
 18  snomed_8      34 non-null     object
 19  snomed

In [10]:
visits_ds = visits_ds[["patient_id", "treatment_start", "treatment_period", "visit_id", "vstart", "vprovider","care_site"]].copy()
visits_ds = visits_ds.drop_duplicates()
visits_ds.shape

(5596, 7)

In [11]:
visits_ds = pd.merge(visits_ds, wide, on="visit_id")
visits_ds.head()

,patient_id,treatment_start,treatment_period,visit_id,vstart,vprovider,care_site,diagnosis_1,diagnosis_2,diagnosis_3,...,icd9_2,icd9_3,icd9_4,icd9_5,icd9_6,icd9_7,icd9_8,icd9_9,icd9_10,icd9_11
0,111598,2008-01-15,90,5351372,2008-01-28,50058.0,11234.0,Vomiting,Epidemic vertigo,None,...,78703,None,None,None,None,None,None,None,None,None
1,111598,2008-01-15,90,5351385,2008-01-28,315573.0,11283.0,Atrial fibrillation,Paroxysmal supraventricular tachycardia,None,...,42731,None,None,None,None,None,None,None,None,None
2,111598,2008-01-15,90,5351386,2008-01-31,218450.0,11234.0,Atrial fibrillation,Premature beats,Type 2 diabetes mellitus,...,42760,42731,None,None,None,None,None,None,None,None
3,111598,2008-01-15,90,5351403,2008-02-05,382137.0,28276.0,Angina decubitus,Mitral and aortic stenosis,None,...,4130,None,None,None,None,None,None,None,None,None
4,111598,2008-01-15,90,5351324,2008-02-05,447710.0,11222.0,Conduction disorder of the heart,Atrial fibrillation,None,...,42731,None,None,None,None,None,None,None,None,None


In [12]:
visits_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5596 entries, 0 to 5595
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   patient_id        5596 non-null   int64         
 1   treatment_start   5596 non-null   datetime64[ns]
 2   treatment_period  5596 non-null   int64         
 3   visit_id          5596 non-null   int64         
 4   vstart            5596 non-null   datetime64[ns]
 5   vprovider         5587 non-null   float64       
 6   care_site         5591 non-null   float64       
 7   diagnosis_1       5596 non-null   object        
 8   diagnosis_2       4258 non-null   object        
 9   diagnosis_3       2125 non-null   object        
 10  diagnosis_4       1026 non-null   object        
 11  diagnosis_5       439 non-null    object        
 12  diagnosis_6       185 non-null    object        
 13  diagnosis_7       85 non-null     object        
 14  diagnosis_8       34 non

Clean and reshape procedures.

Begin by bringing in CPT descriptions.

In [13]:
procedures_ds.head()

,patient_id,drug_treatment_start,drug_treatment_period,vtype,visit_id,procedure_name,procedure_code,code_type,vstart,vend,vduration,vprovider,care_site
0,111598,2008-01-15,90,PROCEDURE,5351385,NaN,36415,CPT4,2008-01-28,NaT,NaN,315573.0,NaN
1,111598,2008-01-15,90,PROCEDURE,5351372,NaN,99214,CPT4,2008-01-28,NaT,NaN,50058.0,NaN
2,111598,2008-01-15,90,PROCEDURE,5351372,"Bone graft, carpals and metacarpals",78.04,ICD9Proc,2008-01-28,NaT,NaN,50058.0,NaN
3,111598,2008-01-15,90,PROCEDURE,5351403,No matching concept,No matching concept,None,2008-02-05,NaT,NaN,382137.0,NaN
4,111598,2008-01-15,90,PROCEDURE,5351403,No matching concept,No matching concept,None,2008-02-05,NaT,NaN,382137.0,NaN


In [14]:
cpt_codes.head()

,cpt_code,cpt_name
0,86890,Autologous blood process
1,86891,Autologous blood op salvage
2,86927,Plasma fresh frozen
3,86930,Frozen blood prep
4,86931,Frozen blood thaw


In [20]:
cpt_procedures = procedures_ds[procedures_ds["code_type"]=="CPT4"].copy()
cpt_procedures = pd.merge(cpt_procedures, cpt_codes, left_on="procedure_code", right_on="cpt_code")
procedures2 = pd.merge(procedures_ds, cpt_procedures, on=["patient_id", "visit_id"])

procedures2.drop(["vend_x", "vduration_x", "care_site_x", "drug_treatment_start_y", "drug_treatment_period_y", "vtype_y", 
                  "procedure_name_y", "procedure_code_y", "code_type_y", "vstart_y", "vend_y", "vduration_y", "vprovider_y", 
                  "care_site_y", "cpt_code"], axis=1, inplace=True)

procedures2.columns = ["patient_id", "treatment_start", "treatment_period", "vtype", "visit_id", "procedure_name", "procedure_code",
                       "code_type", "vstart", "vprovider", "cpt_name"]

procedures2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45206 entries, 0 to 45205
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   patient_id        45206 non-null  int64         
 1   treatment_start   45206 non-null  datetime64[ns]
 2   treatment_period  45206 non-null  int64         
 3   vtype             45206 non-null  object        
 4   visit_id          45206 non-null  int64         
 5   procedure_name    11775 non-null  object        
 6   procedure_code    45206 non-null  object        
 7   code_type         45206 non-null  object        
 8   vstart            45206 non-null  datetime64[ns]
 9   vprovider         44975 non-null  float64       
 10  cpt_name          45206 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(3), object(5)
memory usage: 4.1+ MB


In [21]:
procedures_ds = procedures2.copy()
procedures_ds.head()

,patient_id,treatment_start,treatment_period,vtype,visit_id,procedure_name,procedure_code,code_type,vstart,vprovider,cpt_name
0,111598,2008-01-15,90,PROCEDURE,5351385,NaN,36415,CPT4,2008-01-28,315573.0,Routine venipuncture
1,111598,2008-01-15,90,PROCEDURE,5351372,NaN,99214,CPT4,2008-01-28,50058.0,established patient office visit
2,111598,2008-01-15,90,PROCEDURE,5351372,"Bone graft, carpals and metacarpals",78.04,ICD9Proc,2008-01-28,50058.0,established patient office visit
3,111598,2008-01-15,90,PROCEDURE,5351324,NaN,99291,CPT4,2008-02-05,447710.0,Routine venipuncture
4,111598,2008-01-15,90,PROCEDURE,5351324,NaN,99291,CPT4,2008-02-05,447710.0,critical care visit


Align and 'explode' procedure codes, code types and procedure descriptions.

In [22]:
procedures2 = procedures_ds[["visit_id", "code_type", "procedure_code", "procedure_name", "cpt_name"]].copy()

procedures2.loc[procedures2["procedure_code"]=="None", "procedure_code"] = np.nan
procedures2.loc[procedures2["procedure_code"]=="No matching concept", "procedure_code"] = np.nan
procedures2.loc[procedures2["code_type"]=="None", "code_type"] = np.nan
procedures2.loc[procedures2["procedure_name"]=="No matching concept", "procedure_name"] = np.nan

procedures2["procedure_name"] = procedures2["procedure_name"].fillna('')
procedures2["cpt_name"] = procedures2["cpt_name"].fillna('')
procedures2["code_type"] = procedures2["code_type"].fillna('')

procedures2 = procedures2.groupby(['visit_id', 'procedure_code'], as_index = False).agg({'code_type': lambda x: ';'.join(x), 
                                                                                       'procedure_name': lambda x: ';'.join(set(x)),
                                                                                       'cpt_name': lambda x: '; '.join(set(x)) })

procedures2["description"] = procedures2["procedure_name"] + procedures2["cpt_name"]
procedures2.drop(["procedure_name", "cpt_name"], axis=1, inplace=True, errors="ignore")

procedures2.head()

,visit_id,procedure_code,code_type,description
0,44245,94640,CPT4,Airway inhalation treatment
1,44246,36415,CPT4,Routine venipuncture
2,44260,99334,CPT4,"home health care, established patient"
3,44262,88305,CPT4,Surgical pathology
4,44270,92012,CPT4;CPT4,FUNDUS PHOTOGRAPHY WITH INTERPRETATION AND REP...


In [24]:
grouped_p = procedures2.groupby(['visit_id'], as_index = False).agg({'procedure_code': lambda x: '~'.join(x), 
                                                                'code_type': lambda x: '~'.join(x),
                                                                'description': lambda x: '~'.join(x)})
long_p = {}
long_p[0] = grouped_p['procedure_code'].str.split(pat="~", expand=True)
long_p[1] = grouped_p['code_type'].str.split(pat="~",expand=True)
long_p[2] = grouped_p['description'].str.split(pat="~",expand=True)
wide_p = pd.concat(long_p, axis=1)
wide_p["visit_id"] = grouped_p["visit_id"].copy()


wide_p.columns = ["pc_1", "pc_2", "pc_3", "pc_4", "pc_5", "pc_6", "pc_7", "pc_8", "pc_9", "pc_10", "pc_11", "pc_12", "pc_13", "pc_14", 
                  "pc_15", "pc_16", "pct_1", "pct_2", "pct_3", "pct_4", "pct_5", "pct_6", "pct_7", "pct_8", "pct_9", "pct_10", "pct_11", 
                  "pct_12", "pct_13", "pct_14", "pct_15", "pct_16", "pdesc_1", "pdesc_2", "pdesc_3", "pdesc_4", "pdesc_5", 
                  "pdesc_6", "pdesc_7", "pdesc_8", "pdesc_9", "pdesc_10", "pdesc_11", "pdesc_12", "pdesc_13", 
                  "pdesc_14", "pdesc_15", "pdesc_16", "visit_id"]

wide_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4730 entries, 0 to 4729
Data columns (total 49 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pc_1      4730 non-null   object
 1   pc_2      3093 non-null   object
 2   pc_3      1618 non-null   object
 3   pc_4      711 non-null    object
 4   pc_5      323 non-null    object
 5   pc_6      135 non-null    object
 6   pc_7      68 non-null     object
 7   pc_8      41 non-null     object
 8   pc_9      22 non-null     object
 9   pc_10     11 non-null     object
 10  pc_11     8 non-null      object
 11  pc_12     5 non-null      object
 12  pc_13     4 non-null      object
 13  pc_14     4 non-null      object
 14  pc_15     2 non-null      object
 15  pc_16     1 non-null      object
 16  pct_1     4730 non-null   object
 17  pct_2     3093 non-null   object
 18  pct_3     1618 non-null   object
 19  pct_4     711 non-null    object
 20  pct_5     323 non-null    object
 21  pct_6     135 

In [25]:
wide_p.shape

(4730, 49)

Clean procedure types, so each type is listed once per field.

In [26]:
code_types = ["pct_1", "pct_2", "pct_3", "pct_4", "pct_5", "pct_6", "pct_7", "pct_8", "pct_9", "pct_10", "pct_11", 
       "pct_12", "pct_13", "pct_14", "pct_15", "pct_16"]

for r in range(0, len(wide_p)):
    for c in code_types:
        if wide_p.at[r, c] != None:
            if len(wide_p.at[r, c].split(";")) > 1:
                wide_p.at[r, c] = wide_p.at[r, c].split(";")[0]
                if isinstance(wide_p.at[r, c], (list, tuple)):
                     wide_p.at[r, c] = wide_p.at[r, c][0]
        

Combine procedure code type, code and description into a single field for each procedure code associated with a visit.

In [27]:
desc_cols = ["pdesc_1", "pdesc_2", "pdesc_3", "pdesc_4", "pdesc_5", "pdesc_6", "pdesc_7", "pdesc_8", "pdesc_9", "pdesc_10", "pdesc_11", 
             "pdesc_12", "pdesc_13", "pdesc_14", "pdesc_15", "pdesc_16"]

for r in range(0, len(wide_p)):
    count = 1
    for d in desc_cols:
        c1 = "pc_" + str(count)
        c2 = "pct_" + str(count) 
        if wide_p.at[r, d] != None:
            if  wide_p.at[r, c2] != None:
                if wide_p.at[r, c1] != None:
                    wide_p.at[r, d] = wide_p.at[r, c2] + ": " + wide_p.at[r, c1] + " " + wide_p.at[r, d]
                else:
                    wide_p.at[r, d] = wide_p.at[r, c2] + ": " + wide_p.at[r, d]
            else:
                 wide_p.at[r, d] = wide_p.at[r, c1] + ": " + wide_p.at[r, d]
        else:
            pass
        count += 1
    
wide_p.head()

,pc_1,pc_2,pc_3,pc_4,pc_5,pc_6,pc_7,pc_8,pc_9,pc_10,...,pdesc_8,pdesc_9,pdesc_10,pdesc_11,pdesc_12,pdesc_13,pdesc_14,pdesc_15,pdesc_16,visit_id
0,94640,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,44245
1,36415,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,44246
2,99334,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,44260
3,88305,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,44262
4,92012,92250,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,44270


Now that the procedure information is combined, drop the unnecessary columns.

In [28]:
drop_cols = ["pc_1", "pc_2", "pc_3", "pc_4", "pc_5", "pc_6", "pc_7", "pc_8", "pc_9", "pc_10", "pc_11", "pc_12", "pc_13", "pc_14",
             "pc_15", "pc_16", "pct_1", "pct_2", "pct_3", "pct_4", "pct_5", "pct_6", "pct_7", "pct_8", "pct_9", "pct_10", "pct_11", 
             "pct_12", "pct_13", "pct_14", "pct_15", "pct_16"]

wide_p.drop(drop_cols, inplace=True, axis=1, errors="ignore")

In [29]:
wide_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4730 entries, 0 to 4729
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pdesc_1   4730 non-null   object
 1   pdesc_2   3093 non-null   object
 2   pdesc_3   1618 non-null   object
 3   pdesc_4   711 non-null    object
 4   pdesc_5   323 non-null    object
 5   pdesc_6   135 non-null    object
 6   pdesc_7   68 non-null     object
 7   pdesc_8   41 non-null     object
 8   pdesc_9   22 non-null     object
 9   pdesc_10  11 non-null     object
 10  pdesc_11  8 non-null      object
 11  pdesc_12  5 non-null      object
 12  pdesc_13  4 non-null      object
 13  pdesc_14  4 non-null      object
 14  pdesc_15  2 non-null      object
 15  pdesc_16  1 non-null      object
 16  visit_id  4730 non-null   int64 
dtypes: int64(1), object(16)
memory usage: 628.3+ KB


Flatten the procedures data frame.

In [30]:
procedures_ds = procedures_ds[["patient_id", "treatment_start", "treatment_period", "vtype", "visit_id", "vstart", "vprovider"]].copy()
procedures_ds  = procedures_ds .drop_duplicates()
procedures_ds.shape

(4734, 7)

Merge it with the detailed procedure information.

In [31]:
procedures_ds = pd.merge(procedures_ds, wide_p, on="visit_id", how="left")
procedures_ds.head()

,patient_id,treatment_start,treatment_period,vtype,visit_id,vstart,vprovider,pdesc_1,pdesc_2,pdesc_3,...,pdesc_7,pdesc_8,pdesc_9,pdesc_10,pdesc_11,pdesc_12,pdesc_13,pdesc_14,pdesc_15,pdesc_16
0,111598,2008-01-15,90,PROCEDURE,5351385,2008-01-28,315573.0,CPT4: 36415 Routine venipuncture,None,None,...,None,None,None,None,None,None,None,None,None,None
1,111598,2008-01-15,90,PROCEDURE,5351372,2008-01-28,50058.0,"ICD9Proc: 78.04 Bone graft, carpals and metaca...",CPT4: 99214 established patient office visit,None,...,None,None,None,None,None,None,None,None,None,None
2,111598,2008-01-15,90,PROCEDURE,5351324,2008-02-05,447710.0,CPT4: 36415 Routine venipuncture; critical car...,CPT4: 90935 Routine venipuncture; critical car...,CPT4: 99291 Routine venipuncture; critical car...,...,None,None,None,None,None,None,None,None,None,None
3,111598,2008-01-15,90,PROCEDURE,5351415,2008-02-08,449900.0,"CPT4: 00142 Anesthesia, head",None,None,...,None,None,None,None,None,None,None,None,None,None
4,111598,2008-01-15,90,PROCEDURE,5351333,2008-02-13,50055.0,ICD9Proc: 22.6 Other nasal sinusectomyestablis...,CPT4: 99213 established patient office visit,None,...,None,None,None,None,None,None,None,None,None,None


In [32]:
visits_ds.shape

(5596, 40)

Combine procedure information with visits for the rows that have procedure information.

In [33]:
visits_procedures_ds = pd.merge(visits_ds, procedures_ds, on="visit_id", how="left")
visits_procedures_ds.head()

,patient_id_x,treatment_start_x,treatment_period_x,visit_id,vstart_x,vprovider_x,care_site,diagnosis_1,diagnosis_2,diagnosis_3,...,pdesc_7,pdesc_8,pdesc_9,pdesc_10,pdesc_11,pdesc_12,pdesc_13,pdesc_14,pdesc_15,pdesc_16
0,111598,2008-01-15,90,5351372,2008-01-28,50058.0,11234.0,Vomiting,Epidemic vertigo,None,...,None,None,None,None,None,None,None,None,None,None
1,111598,2008-01-15,90,5351385,2008-01-28,315573.0,11283.0,Atrial fibrillation,Paroxysmal supraventricular tachycardia,None,...,None,None,None,None,None,None,None,None,None,None
2,111598,2008-01-15,90,5351386,2008-01-31,218450.0,11234.0,Atrial fibrillation,Premature beats,Type 2 diabetes mellitus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111598,2008-01-15,90,5351403,2008-02-05,382137.0,28276.0,Angina decubitus,Mitral and aortic stenosis,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,111598,2008-01-15,90,5351324,2008-02-05,447710.0,11222.0,Conduction disorder of the heart,Atrial fibrillation,None,...,None,None,None,None,None,None,None,None,None,None


In [34]:
visits_procedures_ds.to_csv("visits_procedures_wide.csv")

Reimport the file after row-title cleanup in Excel.

In [35]:
visits_procedures_ds = pd.read_csv("visits_procedures_wide.csv", parse_dates=["treatment_start", "vstart"])
visits_procedures_ds.head()

,patient_id,treatment_start,treatment_period,visit_id,vstart,vprovider,care_site,diagnosis_1,diagnosis_2,diagnosis_3,...,pdesc_7,pdesc_8,pdesc_9,pdesc_10,pdesc_11,pdesc_12,pdesc_13,pdesc_14,pdesc_15,pdesc_16
0,111598,2008-01-15,90,5351372,2008-01-28,50058.0,11234.0,Vomiting,Epidemic vertigo,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111598,2008-01-15,90,5351385,2008-01-28,315573.0,11283.0,Atrial fibrillation,Paroxysmal supraventricular tachycardia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111598,2008-01-15,90,5351386,2008-01-31,218450.0,11234.0,Atrial fibrillation,Premature beats,Type 2 diabetes mellitus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111598,2008-01-15,90,5351403,2008-02-05,382137.0,28276.0,Angina decubitus,Mitral and aortic stenosis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,111598,2008-01-15,90,5351324,2008-02-05,447710.0,11222.0,Conduction disorder of the heart,Atrial fibrillation,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
visits_procedures_ds.shape

(5600, 56)

There are 5,600 total visits for the 203-person cohort.

Now, we'll look at conditions information to determine which of these visits happened during a condition period for the 97 people who had a code for a lung, liver or thyroid-related disorder following amiodarone therapy.

In [37]:
conditions_ds.shape

(307, 12)

In [38]:
conditions_ds.head()

,patient_id,treatment_start,visit_id,ICD9CODE,ICD9_name,snomed_name,condition_type,snomed_code,condition_start_date,condition_end_date,condition_interval_days,treatment_condition_interval_days
0,11392,2008-06-08,552989,242.9,Thyrotoxicosis without mention of goiter or ot...,Thyrotoxicosis without goiter OR other cause,THYROID,89719007,2008-06-16,2008-06-16,0,8
1,18427,2009-08-04,893707,242.9,Thyrotoxicosis without mention of goiter or ot...,Thyrotoxicosis without goiter OR other cause,THYROID,89719007,2010-10-02,2010-10-02,0,424
2,30161,2008-05-07,1457808,242.9,Thyrotoxicosis without mention of goiter or ot...,Thyrotoxicosis without goiter OR other cause,THYROID,89719007,2010-06-08,2010-06-08,0,762
3,38696,2009-09-15,1871971,242.9,Thyrotoxicosis without mention of goiter or ot...,Thyrotoxicosis without goiter OR other cause,THYROID,89719007,2010-10-13,2010-10-13,0,393
4,38696,2009-09-15,1871971,242.9,Thyrotoxicosis without mention of goiter or ot...,Thyrotoxicosis without goiter OR other cause,THYROID,89719007,2010-10-13,2010-10-13,0,393


In [39]:
conditions_ds[conditions_ds["condition_interval_days"]>0].shape

(33, 12)

There are 307 visits associated with a flagged condition that happened after amiodarone therapy began.  Only 33 of these visits were about a condition recorded as lasting > 1 day.

In [41]:
common_patient_ids = pd.Series(np.intersect1d(conditions_ds["patient_id"], visits_procedures_ds["patient_id"]))
common_patient_ids

0       4549
1       5680
2       5894
3       7134
4       7630
       ...  
92    111598
93    113140
94    114821
95    115881
96    116232
Length: 97, dtype: int64

In [40]:
common_visit_ids = pd.Series(np.intersect1d(conditions_ds["visit_id"], visits_procedures_ds["visit_id"]))
common_visit_ids

0       224884
1       279377
2       289319
3       289337
4       347276
        ...   
251    5555700
252    5555705
253    5573746
254    5573775
255    5573841
Length: 256, dtype: int64

97 patients had one of the sentinel conditions, and there were 256 common visit ids.

In [42]:
condition_visits_ds = pd.merge(visits_procedures_ds, conditions_ds, on=["patient_id", "visit_id"], how="left")
condition_visits_ds = condition_visits_ds.drop_duplicates()
condition_visits_ds.shape

(5617, 66)

In [43]:
condition_visits_ds.to_csv("conditions_visits_procedures.csv")

Reimport the file after row-title cleanup in Excel.

In [48]:
condition_visits_ds = pd.read_csv("conditions_visits_procedures.csv", parse_dates=["treatment_start", "vstart"])
condition_visits_ds.head()

,patient_id,treatment_start,treatment_period,visit_id,vstart,vprovider,care_site,diagnosis_1,diagnosis_2,diagnosis_3,...,pdesc_16,sentinel_condition_code,sentinel_condition_name,sentinel_snomed_name,sentinel_condition_type,sentinel_snomed_code,condition_start_date,condition_end_date,condition_interval_days,treatment_condition_interval_days
0,111598,2008-01-15,90,5351372,2008-01-28,50058.0,11234.0,Vomiting,Epidemic vertigo,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111598,2008-01-15,90,5351385,2008-01-28,315573.0,11283.0,Atrial fibrillation,Paroxysmal supraventricular tachycardia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111598,2008-01-15,90,5351386,2008-01-31,218450.0,11234.0,Atrial fibrillation,Premature beats,Type 2 diabetes mellitus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111598,2008-01-15,90,5351403,2008-02-05,382137.0,28276.0,Angina decubitus,Mitral and aortic stenosis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,111598,2008-01-15,90,5351324,2008-02-05,447710.0,11222.0,Conduction disorder of the heart,Atrial fibrillation,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
condition_visits_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5617 entries, 0 to 5616
Data columns (total 65 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   patient_id                         5617 non-null   int64         
 1   treatment_start                    5617 non-null   datetime64[ns]
 2   treatment_period                   5617 non-null   int64         
 3   visit_id                           5617 non-null   int64         
 4   vstart                             5617 non-null   datetime64[ns]
 5   vprovider                          5608 non-null   float64       
 6   care_site                          5612 non-null   float64       
 7   diagnosis_1                        5617 non-null   object        
 8   diagnosis_2                        4279 non-null   object        
 9   diagnosis_3                        2137 non-null   object        
 10  diagnosis_4                        1